# Неделя 7: NLP

## Генерация текста с помощью модели **GPT**

https://www.kaggle.com/tuckerarrants/text-generation-with-huggingface-gpt2

In [13]:
pip install transformers

Note: you may need to restart the kernel to use updated packages.


In [14]:
pip install gradio

Note: you may need to restart the kernel to use updated packages.


In [22]:
import numpy as np
import pandas as pd
import re
import random

import torch
from tqdm.notebook import tqdm
import transformers
from torch.optim import AdamW
from transformers import AutoModelForCausalLM, AutoTokenizer

#pip install sentencepiece 

device = 'cuda' if torch.cuda.is_available() else 'cpu'


In [23]:
# Загружаем файл с текстом, на котором будем дообучать модель
import re
with open('/kish_lyrics.txt', encoding='utf8') as f:
    text = f.read()

text = re.sub('\n{2,}', '\n', text)
print(text[:100])

 Будь проще и говори на понятном для людей языке. Потому что очень многие политики стараются говорит


In [30]:
from transformers import GPT2Tokenizer

In [31]:
tokenizer = GPT2Tokenizer.from_pretrained('sberbank-ai/rugpt3small_based_on_gpt2')

In [32]:

# токенизируем текст
tokens = tokenizer.encode(text, add_special_tokens=True)
tokens = np.array(tokens)
print(len(tokens))
tokens[:10]

3565


array([12759,  8622,   289,   998,   309, 18848,   547,   541,  1512,
        6718])

In [33]:
# разбиваем на train и test

l = len(tokens)//15
train = []
test = []
for i in range(15):
    if i%5 > 0:
        train.extend(tokens[i*l: (i+1)*l])
    else:
        test.extend(tokens[i*l: (i+1)*l])
train = np.array(train)
test = np.array(test)

print(len(tokens), len(train), len(test))

3565 2844 711


In [34]:
from transformers import GPT2LMHeadModel



# Эту модель просто подгружаем и не будем дообучать 
model_init = GPT2LMHeadModel.from_pretrained(
    'sberbank-ai/rugpt3small_based_on_gpt2',
    output_attentions = False,
    output_hidden_states = False,
)


# Эту модель подгрузим и далее обучим
model = GPT2LMHeadModel.from_pretrained(
    'sberbank-ai/rugpt3small_based_on_gpt2',
    output_attentions = False,
    output_hidden_states = False,
)

model.to(device);
model_init.to(device);

In [16]:
batch_size = 5
max_len = 75
epochs = 10

n_train = len(train)//(batch_size*max_len)
n_test = len(test)//(batch_size*max_len)
print(n_train, n_test)

# устанавливаем оптимизатор
optimizer = AdamW(model.parameters(), lr = 1e-5, eps = 1e-8)

# трансформеры с трудом обучаются, для них нужны разные способы повышения 
# эффективности градиентного спуска
total_steps = n_train * epochs
scheduler = transformers.get_linear_schedule_with_warmup(optimizer, 
                                            num_warmup_steps = 0,
                                            num_training_steps = total_steps)


def accuracy(y_true, logits):
    return torch.mean((y_true[1:] == torch.argmax(logits, dim=2)[:-1]).float()).detach().cpu().numpy()

7 1


In [20]:
# готовим тензоры для обучения

def prep_tensors(x, i, batch_size=batch_size, max_len=max_len):
    batch_ids = x[i*batch_size*max_len: (i+1)*batch_size*max_len]
    batch_ids = batch_ids.reshape(batch_size, max_len)
    batch_ids = torch.tensor(batch_ids).to(device)
    return batch_ids


# обучающий цикл
for epoch in range(1, epochs+1):
    print(f'epoch {epoch}/{epochs} : training')

    train_loss = []
    train_acc = []
    model.train()
    pbar = tqdm(range(n_train))
    for i in pbar:
        batch_ids = prep_tensors(train, i)

        model.zero_grad()
        loss, logits, _ = model(batch_ids,
                                token_type_ids=None, 
                                
                                labels=batch_ids
                             ).values()

        loss.backward()
        torch.nn.utils.clip_grad_norm_(model.parameters(), 1.0)
        optimizer.step()
        scheduler.step()
        
        train_loss.append(loss.item())
        train_acc.append(accuracy(batch_ids, logits))
        pbar.set_description(f'acc {np.mean(train_acc):.4f} loss {np.mean(train_loss):.4f}', refresh=True)

    
    print(f'epoch {epoch}/{epochs} : validation')
    model.eval()
    val_acc = []
    val_loss = []
    pbar = tqdm(range(n_test))
    for i in pbar:
        batch_ids = prep_tensors(test, i)
        with torch.no_grad():        
            loss, logits, _ = model(batch_ids, 
                                token_type_ids=None, 
                                # attention_mask=batch_mask,
                                labels=batch_ids
                                 ).values()
        
        val_loss.append(loss.item())
        val_acc.append(accuracy(batch_ids, logits))
        pbar.set_description(f'acc {np.mean(val_acc):.4f} loss {np.mean(val_loss):.4f}', refresh=True)


epoch 1/10 : training


  0%|          | 0/7 [00:00<?, ?it/s]

epoch 1/10 : validation


  0%|          | 0/1 [00:00<?, ?it/s]

epoch 2/10 : training


...

  0%|          | 0/1 [00:00<?, ?it/s]

epoch 10/10 : training


  0%|          | 0/7 [00:00<?, ?it/s]

epoch 10/10 : validation


  0%|          | 0/1 [00:00<?, ?it/s]

In [21]:
import textwrap

In [22]:
# https://huggingface.co/transformers/main_classes/model.html#transformers.generation_utils.GenerationMixin.generate
# модель без дообучения

# prompt = "print('Hello world!') "
prompt = 'Хочу Вам сказать следующее:'
prompt = tokenizer.encode(prompt, return_tensors='pt').to(device)
out = model_init.generate(
    input_ids=prompt,
    max_length=50,
    num_beams=5,
    do_sample=True,
    temperature=10.,
    top_k=20,
    top_p=0.8,
    no_repeat_ngram_size=3,
    num_return_sequences=7,
    ).cpu().numpy()
for out_ in out:
    print(textwrap.fill(tokenizer.decode(out_), 120), end='\n------------------\n')

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


Хочу Вам сказать следующее:   Если вы хотите стать счастливым в любви к женщине и к ребёнку, нужно просто сделать это,
но если вам этого сделать очень сильно мешает Ваша лень или отсутствие мотивации — это будет уже просто катастрофа...
Что бы стать
------------------
Хочу Вам сказать следующее: я в жизни никогда ничего хорошего, плохого мне от меня не хотят, это их работа. Но у них нет
и шанса мне на доброе обращение ко мне в их лице! Я просто не смогу им ответить. Если бы они
------------------
Хочу Вам сказать следующее:   В этой теме мы будем писать об этом и постараемся помочь Вам в развитии навыков, которые
пригодятся вам во многих ситуациях жизни.   Для этого я хочу предложить несколько простых правил, которые будут Вам
очень кстати, если вы
------------------
Хочу Вам сказать следующее: Вы, возможно, читали "Введение", а также в "Двенадцать дней творения".  Если Вам так трудно
читать и писать, Вам стоит начать писать.  И Вы не сможете не написать о своих чувствах к
------------

In [23]:
for out_ in out:
    print(tokenizer.decode(out_))

Хочу Вам сказать следующее: 
 Если вы хотите стать счастливым в любви к женщине и к ребёнку, нужно просто сделать это, но если вам этого сделать очень сильно мешает Ваша лень или отсутствие мотивации — это будет уже просто катастрофа... 
 Что бы стать
Хочу Вам сказать следующее: я в жизни никогда ничего хорошего, плохого мне от меня не хотят, это их работа. Но у них нет и шанса мне на доброе обращение ко мне в их лице! Я просто не смогу им ответить. Если бы они
Хочу Вам сказать следующее: 
 В этой теме мы будем писать об этом и постараемся помочь Вам в развитии навыков, которые пригодятся вам во многих ситуациях жизни. 
 Для этого я хочу предложить несколько простых правил, которые будут Вам очень кстати, если вы
Хочу Вам сказать следующее: Вы, возможно, читали "Введение", а также в "Двенадцать дней творения".  Если Вам так трудно читать и писать, Вам стоит начать писать.  И Вы не сможете не написать о своих чувствах к
Хочу Вам сказать следующее: "Иисуса я вижу только в образе Его; пот

In [24]:
# дообученная модель

prompt = 'Хочу Вам сказать следующее:'
prompt = tokenizer.encode(prompt, return_tensors='pt').to(device)
out = model.generate(
    input_ids=prompt,
    max_length=50,
    num_beams=5,
    do_sample=True,
    temperature=10.,
    top_k=20,
    top_p=0.8,
    no_repeat_ngram_size=2,
    num_return_sequences=7,
    ).cpu().numpy()
for out_ in out:
    print(textwrap.fill(tokenizer.decode(out_), 120), end='\n------------------\n')

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


Хочу Вам сказать следующее: не важно, как долго длится Ваше лечение. И если у вас все хорошо и вы продолжаете двигаться
— это замечательно. Но если Вы хотите сказать: мне необходимо идти — идите, и я готов пойти… В жизни есть
------------------
Хочу Вам сказать следующее: мы с детства не знали страха перед огнем и всегда, в самые страшные минуты, мы бежали в
темноту, чтобы спасти близких, и тогда мы были готовы. Если ты знаешь врага, не можешь знать, с ним не
------------------
Хочу Вам сказать следующее: если бы не Вы, мир давно бы потерял Украину, не стало той Украины. Я бы сейчас хотел Вам
задать несколько вопросов: – В какой валюте живут украинские пенсионеры сегодня, и когда они уже могут получить пенсию?
------------------
Хочу Вам сказать следующее: когда человек говорит, он должен знать, где и как именно ему следует искать правду и что
нужно говорить людям, которых он любит». А сейчас у вас есть шанс найти своего любимого, и он обязательно вернется. Но
помните
-----------------

In [25]:
# def generate(prompt, len_gen=20, temperature=1):
#     generated = tokenizer.encode(prompt)
#     context = torch.tensor([generated]).to(device)
#     past = None

#     for i in tqdm(range(len_gen)):
#         output, past = model(context, past_key_values=past).values()
#         # token = torch.argmax(output[..., -1, :], dim=-1)
#         output = output / temperature
#         token = torch.distributions.Categorical(logits=output[..., -1, :]).sample()
        
#         generated += token.tolist()
#         context = token.unsqueeze(0)

#     sequence = tokenizer.decode(generated)

#     return sequence

In [26]:
import gradio as gr
from transformers import AutoModelForCausalLM, AutoTokenizer

In [27]:
tokenizer = AutoTokenizer.from_pretrained("microsoft/DialoGPT-medium")
model = AutoModelForCausalLM.from_pretrained("microsoft/DialoGPT-medium")

In [28]:
def predict(input, history=[]):
    # tokenize the new input sentence
    new_user_input_ids = tokenizer.encode(input + tokenizer.eos_token, return_tensors='pt')

    # append the new user input tokens to the chat history
    bot_input_ids = torch.cat([torch.LongTensor(history), new_user_input_ids], dim=-1)

    # generate a response 
    history = model.generate(bot_input_ids, max_length=1000, pad_token_id=tokenizer.eos_token_id).tolist()

    # convert the tokens to text, and then split the responses into lines
    response = tokenizer.decode(history[0]).split("<|endoftext|>")
    response = [(response[i], response[i+1]) for i in range(0, len(response)-1, 2)]  # convert to tuples of list
    return response, history

gr.Interface(fn=predict,
             inputs=["text", "state"],
             outputs=["chatbot", "state"]).launch(debug=False)

Colab notebook detected. To show errors in colab notebook, set `debug=True` in `launch()`
Running on public URL: https://53796.gradio.app

This share link expires in 72 hours. For free permanent hosting, check out Spaces: https://huggingface.co/spaces


(<gradio.routes.App at 0x7fcabb31a710>,
 'http://127.0.0.1:7862/',
 'https://53796.gradio.app')